In [38]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib.colors import ListedColormap
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


In [39]:
def get_features_names(fileName, test=False):
    features_names = np.array([],str)    
    with open(fileName, 'r') as f:
        reader = csv.DictReader(f, delimiter=',')
        row = next(reader)

        for col in row:     
            features_names = np.append(features_names,col)
        
        if test == True:
            features_names = features_names[1:]
        else:
            features_names = features_names[1:-1]
            
    return features_names             

In [40]:
feat_train = get_features_names('../dataset/train.csv',False)
feat_test = get_features_names('../dataset/test.csv',True)
    
if False:   
    print(feat_train)
    print(feat_test)
    
    for c in range(len(feat_train)):
        if feat_train[c] != feat_test[c]:
            print("NON")

In [41]:
features = get_features_names('../dataset/train.csv',False)

In [42]:
def read_train(fileName):
    data = []
    target = np.array([],float)
    features_names = np.array([],str)
    genders = np.array([],str)
    races = np.array([],str)
    fields = np.array([],str)
    
    with open(fileName, 'r') as f:
        reader = csv.DictReader(f, delimiter=',')
        first = True
        for row in reader:
            if first == True:
                for col in row:     
                    features_names = np.append(features_names,col)
            
                features_names = features_names[1:-1]
                first = False
            else:
                r = np.array([],float)
                for col in range(len(features_names)):
                    try:
                        val = float(row[features_names[col]])
                        r = np.append(r,val)
                    except ValueError:
                        val = str(row[features_names[col]])
                        val = val.lower()
                        val = val.strip()

                        if  features_names[col] == 'gender':
                            if val not in genders:
                                genders = np.append(genders,val)

                            i, = np.where(genders == val)
                            val = float(i[0])

                        if  features_names[col] == 'race' or features_names[col] == 'race_o':
                            if val not in races:
                                races = np.append(races,val)

                            i, = np.where(races == val)
                            val = float(i[0])

                        if  features_names[col] == 'field':
                            if val not in fields:
                                fielsds = np.append(fields,val)

                            i, = np.where(fielsds == val)
                            val = float(i[0])

                        if type(val) is str:
                            val = float(0.0)  # TODO field empty ....
                            
                        r = np.append(r,val)

                data.append(r)
                target = np.append(target,row['match'])
                
    return np.array(data, float), target, features_names, genders, races, fields              

In [43]:
def read_test(fileName):
    data = []
    features_names = np.array([],str)
    genders = np.array([],str)
    races = np.array([],str)
    fields = np.array([],str)
    ids = []
    
    with open(fileName, 'r') as f:
        reader = csv.DictReader(f, delimiter=',')
        first = True
        for row in reader:
            if first == True:
                for col in row:     
                    features_names = np.append(features_names,col)
            
                features_names = features_names[1:]
                first = False
            else:
                r = np.array([],float)
                for col in range(len(features_names)):
                    try:
                        val = float(row[features_names[col]])
                        r = np.append(r,val)
                    except ValueError:
                        val = str(row[features_names[col]])
                        val = val.lower()
                        val = val.strip()

                        if  features_names[col] == 'gender':
                            if val not in genders:
                                genders = np.append(genders,val)

                            i, = np.where(genders == val)
                            val = float(i[0])

                        if  features_names[col] == 'race' or features_names[col] == 'race_o':
                            if val not in races:
                                races = np.append(races,val)

                            i, = np.where(races == val)
                            val = float(i[0])

                        if  features_names[col] == 'field':
                            if val not in fields:
                                fielsds = np.append(fields,val)

                            i, = np.where(fielsds == val)
                            val = float(i[0])

                        if type(val) is str:
                            val = float(0.0)  # TODO field empty ....
                            
                        r = np.append(r,val)

                data.append(r)
                ids.append(row['id'])
                
    return np.array(data, float), np.array(ids), features_names, genders, races, fields

In [44]:
X_all, y_all, features_names, genders, races, fields = read_train('../dataset/train.csv')
X_final_test, Ids, features_names_t, genders_t, races_t, fields_t = read_test('../dataset/test.csv')

if False:
    #print(len(X_all))
    #print(len(features_names))
    #print(len(y_all))
    #print(features_names)
    #print(X_all)
    #print(y_all)
    print(genders)
    print(genders_t)
    print(races)
    print(races_t)
    print(len(races))
    print(len(races_t))
    print(fields)
    print(fields_t)
    print(len(fields))
    print(len(fields_t))
    
    
    #print(len(X_final_test))
    #print(len(features_names_t))  
    
if False:
    nb_float = 0
    nb_not_float = 0
    for r in X_all:
        for c in r:
            if type(c) is str:
                nb_not_float += 1
            else:
                nb_float += 1


    print("nb float = %d" % nb_float)
    print("nb_not_float = %d" % nb_not_float)

    print(Ids)
    

In [45]:
def compute_mediane(tab):
    newTab = np.array(tab)
    for col in range(len(newTab[0])):
        vect = []
        for row in range(len(newTab)):
            if newTab[row][col] > -999.0:
                vect.append(newTab[row][col])
        
        med = np.median(vect,0)
        
        for row in range(len(newTab)):
            if newTab[row][col] < -999.0:
                newTab[row][col] = med
                
    return newTab      

In [46]:
def tree_to_code(this_tree, feature_names):
    from sklearn.tree import _tree
    tree = this_tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree.feature
    ]
    print("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree.children_right[node], depth + 1)
        else:
            print("{}return {}".format(indent, tree.value[node]))

    recurse(0, 1)

In [47]:
# Split datas in 2 parts (80% for trainning/validation and 20% for test)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42, shuffle=True, stratify=y_all)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(6381, 61)
(6381,)
(1596, 61)
(1596,)


In [48]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train, sample_weight=None, check_input=True, X_idx_sorted=None)

if False:
    tree_to_code(clf, features_names)

In [49]:
if False:
    print(X_test)
    print(y_test)

In [50]:
y_pred = clf.predict(X_test)
    
clf.score(X_test, y_test, sample_weight=None)    

0.80952380952380953

In [51]:
print(len(X_final_test))
print(len(Ids))

399
399


In [52]:
prediction = clf.predict(X_final_test)

In [53]:
print(len(prediction))
print(len(Ids))

399
399


In [54]:
prediction = np.insert(prediction,0,0)
Ids = np.insert(Ids,0,101)

if False:
    print(len(prediction))
    print(len(Ids))
    print(prediction)
    print(Ids)

In [55]:
with open('Sueur_Fuchs_Pont_01.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'match']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for r in range(len(prediction)):
        writer.writerow({'id': Ids[r], 'match': prediction[r]})

In [57]:
# Iteration 2
X_final_test_median = compute_mediane(X_final_test)
X_all_median = compute_mediane(X_all)

if False:
    for r in range(len(X_final_test_median)):
        for c in range(len(X_final_test_median[0])):
            if X_final_test_median[r][c] != X_final_test[r][c]:
                print("Different")


In [58]:
# Split datas in 2 parts (80% for trainning/validation and 20% for test)
X_train_median, X_test_median, y_train_median, y_test_median = train_test_split(X_all_median, y_all, test_size=0.2, random_state=42, shuffle=True, stratify=y_all)

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train_median, y_train_median, sample_weight=None, check_input=True, X_idx_sorted=None)

y_pred_median = clf.predict(X_test_median)
clf.score(X_test_median, y_test_median, sample_weight=None) 


0.81390977443609025

In [62]:
prediction_median = clf.predict(X_final_test_median)
prediction_median = np.insert(prediction_median,0,0)

In [63]:
with open('Sueur_Fuchs_Pont_02.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'match']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for r in range(len(prediction_median)):
        writer.writerow({'id': Ids[r], 'match': prediction_median[r]})